In [ ]:
import os

import numpy as np
import pandas as pd
import spikeextractors as se
import spiketoolkit as st
import spikewidgets as sw

import tqdm.notebook as tqdm
from scipy.signal import periodogram, spectrogram

import matplotlib.pyplot as plt
# %matplotlib inline
# %config InlineBackend.figure_format='retina'

import holoviews as hv
import holoviews.operation.datashader
import holoviews.operation.timeseries


hv.extension("bokeh")

import panel as pn
import panel.widgets as pnw

pn.extension()

from LoisLFPutils.utils import *

In [ ]:
# Path to the data folder in the repo

data_path = r""




# !!! start assign jupyter notebook parameter(s) !!!

data_path = '2021-02-22_22-00-20_Or179_or177_overnight'

# !!! end assign jupyter notebook parameter(s) !!!

In [ ]:
data_path = os.path.join('../../../../data/',data_path)

# Path to the raw data in the hard drive
with open(os.path.normpath(os.path.join(data_path, 'LFP_location.txt'))) as f:    
    OE_data_path = f.read()

### Get each bird's recording, and their microphone channels

In [ ]:
# This needs to be less repetitive
if 'Or177' in data_path:
    # Whole recording from the hard drive
    recording = se.BinDatRecordingExtractor(OE_data_path,30000,40, dtype='int16')

    # Note I am adding relevant ADC channels
    # First bird
    Or179_recording = se.SubRecordingExtractor(
        recording,
        channel_ids=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11,12,13,14,15, 32])

    # Second bird
    Or177_recording = se.SubRecordingExtractor(
        recording,
        channel_ids=[16, 17,18,19,20,21,22,23,24,25,26,27,28,29,30,31, 33])

    # Bandpass fiter microphone recoridngs
    mic_recording = st.preprocessing.bandpass_filter(
        se.SubRecordingExtractor(recording,channel_ids=[32,33]),
        freq_min=500,
        freq_max=1400
    )
else:
    # Whole recording from the hard drive
    recording = se.BinDatRecordingExtractor(OE_data_path, 30000, 24, dtype='int16')

    # Note I am adding relevant ADC channels
    # First bird
    Or179_recording = se.SubRecordingExtractor(
        recording,
        channel_ids=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11,12,13,14,15,16])

    # Bandpass fiter microphone recoridngs
    mic_recording = st.preprocessing.bandpass_filter(
        se.SubRecordingExtractor(recording,channel_ids=[16]),
        freq_min=500,
        freq_max=1400
    )

In [ ]:
# Get wav files
wav_names = [file_name for file_name in os.listdir(data_path) if file_name.endswith('.wav')]
wav_paths = [os.path.join(data_path,wav_name) for wav_name in wav_names]

# Get tranges for wav files in the actual recording
# OE_data_path actually contains the path all the way to the .bin. We just need the parent directory
# with the timestamp.

# Split up the path
OE_data_path_split= OE_data_path.split(os.sep)

# Take only the first three. os.path is weird so we manually add the separator after the 
# drive name.
OE_parent_path = os.path.join(OE_data_path_split[0] + os.sep, *OE_data_path_split[1:3])

# Get all time ranges given the custom offset.
tranges=np.array([
    get_trange(OE_parent_path, path, offset=datetime.timedelta(seconds=0), duration=3) 
    for path in wav_paths])


In [ ]:
wav_df = pd.DataFrame({'wav_paths':wav_paths, 'wav_names':wav_names, 'trange0':tranges[:, 0], 'trange1':tranges[:, 1]})
wav_df.head()

Connect the wav files to the recording. Manually input to gut check yourself. If it is before 2021 02 21 at 11:00 am PST, you need to add a time delay.

In [ ]:
wav_f,_,_,_=wav_df.loc[0,:]
wav_f, data_path
datetime.datetime(2021,2,23,8,11,1) - datetime.datetime(2021, 2, 22,22,0,20) 

In [ ]:
paths, name, tr0, tr1 = wav_df.loc[0,:]
sw.plot_spectrogram(mic_recording, trange= [tr0,tr1+10], freqrange=[300,4000], nfft=2**10, channel=32)

In [ ]:
np.linspace(0,130,14)

In [ ]:
# Set up widgets
wav_selector = pnw.Select(options=[(i, name) for i, name in enumerate(wav_df.wav_names.values)], name="Select song file")
# offset_selector = pnw.Select(options=np.linspace(-10,10,21).tolist(), name="Select offset")
window_radius_selector = pnw.Select(options=[10,20,30,40,60], name="Select window radius")
spect_chan_selector = pnw.Select(options=list(range(16)), name="Spectrogram channel")
spect_freq_lo = pnw.Select(options=np.linspace(0,130,14).tolist(), name="Low frequency for spectrogram (Hz)")
spect_freq_hi = pnw.Select(options=np.linspace(130,0,14).tolist(), name="Hi frequency for spectrogram (Hz)")
log_nfft_selector = pnw.Select(options=np.linspace(10,16,7).tolist(), value=14, name="magnitude of nfft (starts at 256)")

@pn.depends(
    wav_selector=wav_selector.param.value,
#     offset=offset_selector.param.value,
    window_radius=window_radius_selector.param.value,
    spect_chan=spect_chan_selector.param.value,
    spect_freq_lo=spect_freq_lo.param.value,
    spect_freq_hi=spect_freq_hi.param.value,
    log_nfft=log_nfft_selector.param.value
)
def create_figure(wav_selector, 
#                   offset,
                  window_radius, spect_chan,
                  spect_freq_lo, spect_freq_hi, log_nfft):
    
    # Each column in each row to a tuple that we unpack
    wav_file_path, wav_file_name, tr0, tr1 = wav_df.loc[wav_selector[0],:]
        
    # Set up figure
    fig,axes = plt.subplots(4,1, figsize=(16,12))
    
    # Get wav file numpy recording object
    wav_recording = get_wav_recording(wav_file_path)
    
    # Apply offset and apply window radius
    offset = 0
    tr0 = tr0+ offset-window_radius
    # Add duration of wav file
    tr1 = tr1+ offset+window_radius+wav_recording.get_num_frames()/wav_recording.get_sampling_frequency()
    
    
    '''Plot sound spectrogram (Hi fi mic)'''
    sw.plot_spectrogram(wav_recording, channel=0, freqrange=[300,14000],ax=axes[0])
    axes[0].set_title('Hi fi mic spectrogram')
    
    '''Plot sound spectrogram (Lo fi mic)''' 
    if 'Or179' in wav_file_name:
        LFP_recording = Or179_recording
    elif 'Or177' in wav_file_name:
        LFP_recording = Or177_recording
    
    mic_channel = LFP_recording.get_channel_ids()[-1]
    
    sw.plot_spectrogram(
        mic_recording,
        mic_channel,
        trange=[tr0, tr1],
        freqrange=[600,4000],
        ax=axes[1]
    )
    
    axes[1].set_title('Lo fi mic spectrogram')
    
    '''Plot LFP timeseries'''
    chan_ids = np.array([LFP_recording.get_channel_ids()]).flatten()
    sw.plot_timeseries(
        LFP_recording,
        channel_ids=[chan_ids[spect_chan]],
        trange=[tr0, tr1],
        ax=axes[2]
    )
    axes[2].set_title('Raw LFP')
    
    # Clean lines
    for line in plt.gca().lines:
        line.set_linewidth(0.5)    
    
    '''Plot LFP spectrogram'''
    sw.plot_spectrogram(
        LFP_recording,
        channel=chan_ids[spect_chan],
        freqrange=[spect_freq_lo,spect_freq_hi],
        trange=[tr0, tr1],
        ax=axes[3],
        nfft=int(2**log_nfft)
    )
    axes[3].set_title('LFP')
    
    for i, ax in enumerate(axes):
        ax.set_yticks([ax.get_ylim()[1]])
        ax.set_yticklabels([ax.get_ylim()[1]])
        ax.set_xlabel('')
        
    # Show 30 Hz
    ax.set_yticks([30, ax.get_ylim()[1]])
    ax.set_yticklabels([30, ax.get_ylim()[1]])
            
    
    return fig

dash = pn.Column(
    pn.Row(wav_selector, window_radius_selector,spect_chan_selector),
    pn.Row(spect_freq_lo,spect_freq_hi,log_nfft_selector),
    create_figure
);

dash

## Looking at all channels at a time:

In [ ]:
# Make chanmap
chanmap=np.array([[3, 7, 11, 15],[2, 4, 10, 14],[4, 8, 12, 16],[1, 5, 9, 13]])

# Set up widgets
wav_selector = pnw.Select(options=[(i, name) for i, name in enumerate(wav_df.wav_names.values)], name="Select song file")
window_radius_selector = pnw.Select(options=[10,20,30,40,60], name="Select window radius")
spect_freq_lo = pnw.Select(options=np.linspace(0,130,14).tolist(), name="Low frequency for spectrogram (Hz)")
spect_freq_hi = pnw.Select(options=np.linspace(130,0,14).tolist(), name="Hi frequency for spectrogram (Hz)")
log_nfft_selector = pnw.Select(options=np.linspace(10,16,7).tolist(),value=14, name="magnitude of nfft (starts at 256)")

def housekeeping(wav_selector, window_radius):
        # Each column in each row to a tuple that we unpack
    wav_file_path, wav_file_name, tr0, tr1 = wav_df.loc[wav_selector[0],:]
    
    # Get wav file numpy recording object
    wav_recording = get_wav_recording(wav_file_path)
    
    # Apply offset and apply window radius
    offset = 0
    tr0 = tr0+ offset-window_radius
    
    # Add duration of wav file
    tr1 = tr1+ offset+window_radius+wav_recording.get_num_frames()/wav_recording.get_sampling_frequency()
    
    return wav_recording, wav_file_name, tr0, tr1
    
@pn.depends(
    wav_selector=wav_selector.param.value,
    window_radius=window_radius_selector.param.value)
def create_sound_figure(wav_selector, window_radius):
    
    # Housekeeping
    wav_recording, wav_file_name, tr0, tr1= housekeeping(wav_selector, window_radius)

    # Set up figure for sound
    fig,axes = plt.subplots(1,2, figsize=(16,2))
    
    '''Plot sound spectrogram (Hi fi mic)'''
    sw.plot_spectrogram(wav_recording, channel=0, freqrange=[300,14000], ax=axes[0])
    axes[0].set_title('Hi fi mic spectrogram')
    
    '''Plot sound spectrogram (Lo fi mic)''' 
    if 'Or179' in wav_file_name:
        LFP_recording = Or179_recording
    elif 'Or177' in wav_file_name:
        LFP_recording = Or177_recording
    
    mic_channel = LFP_recording.get_channel_ids()[-1]
    
    sw.plot_spectrogram(
        mic_recording,
        mic_channel,
        trange=[tr0, tr1],
        freqrange=[600,4000],
        ax=axes[1]
    )
    
    axes[1].set_title('Lo fi mic spectrogram')
    for ax in axes:
        ax.axis('off')
    
    return fig

@pn.depends(
    wav_selector=wav_selector.param.value,
    window_radius=window_radius_selector.param.value,
    spect_freq_lo=spect_freq_lo.param.value,
    spect_freq_hi=spect_freq_hi.param.value,
    log_nfft=log_nfft_selector.param.value
)
def create_LFP_figure(wav_selector, window_radius,
                  spect_freq_lo, spect_freq_hi, log_nfft):
    # Housekeeping
    wav_recording, wav_file_name, tr0, tr1 = housekeeping(wav_selector, window_radius)
    
    fig,axes=plt.subplots(4,4,figsize=(16,8))
    
    '''Plot LFP'''
    for i in range(axes.shape[0]):
        for j in range(axes.shape[1]):
            ax = axes[i][j]
            sw.plot_spectrogram(recording, chanmap[i][j], trange=[tr0, tr1],
                                freqrange=[spect_freq_lo,spect_freq_hi],
                                nfft=int(2**log_nfft), ax=ax, cmap='magma')
            ax.axis('off')
            # Set channel as title
            ax.set_title(chanmap[i][j])
    
    # Clean up
    for i in range(axes.shape[0]):
        for j in range(axes.shape[1]):
            ax=axes[i][j]
            
            ax.set_yticks([ax.get_ylim()[1]])
            ax.set_yticklabels([ax.get_ylim()[1]])
            ax.set_xlabel('')

            # Show 30 Hz
            ax.set_yticks([30, ax.get_ylim()[1]])
            ax.set_yticklabels([30, ax.get_ylim()[1]])
            
    
    return fig

dash = pn.Column(
    pn.Row(wav_selector,window_radius_selector),
    pn.Row(spect_freq_lo,spect_freq_hi,log_nfft_selector),
    create_sound_figure, create_LFP_figure
);

# Sleep data analysis!

In [ ]:
csvs = [os.path.normpath(os.path.join(data_path,file)) for file in os.listdir(data_path) if file.endswith('.csv')] 
csvs

In [ ]:
csv = csvs[0]
df = pd.read_csv(csv)
del df['Unnamed: 0']
df.head()

In [ ]:
csv_name = csv.split(os.sep)[-1]
rec=None
if 'Or179' in csv_name:
    rec = Or179_recording
elif 'Or177' in csv_name:
    rec = Or177_recording

# Get second to last element in split
channel = int(csv_name.split('_')[-2])

In [ ]:
window_slider = pn.widgets.DiscreteSlider(
    name='window size',
    options=[*range(1,1000)],
    value=1
)

freq_slider_1 = pn.widgets.DiscreteSlider(
    name='f (Hz)',
    options=[*range(1,200)],
    value=30
)

freq_slider_2 = pn.widgets.DiscreteSlider(
    name='f (Hz)',
    options=[*range(1,200)],
    value=10
)

freq_slider_3 = pn.widgets.DiscreteSlider(
    name='f (Hz)',
    options=[*range(1,200)],
    value=4
)

range_slider = pn.widgets.RangeSlider(
    start=0,
    end=df.t.max(),
    step=10,
    value=(0, 500),
    name="Time range",
    value_throttled=(0,500)
)

@pn.depends(window=window_slider.param.value,
           freq_1=freq_slider_1.param.value,
           freq_2=freq_slider_2.param.value,
           freq_3=freq_slider_3.param.value,
           rang=range_slider.param.value_throttled)
def plot_ts(window, freq_1, freq_2, freq_3, rang):
#     subdf = df.loc[
#         ((df['f']==freq_1)|(df['f']==freq_2)|(df['f']==freq_3)) & (df['t'] < 37800),:]
    subdf = df.loc[
        ((df['f']==freq_1)|(df['f']==freq_2)|(df['f']==freq_3)) 
        & ((df['t'] > rang[0]) & (df['t'] < rang[1])),:]
    
    return hv.operation.timeseries.rolling(
        hv.Curve(
            data = subdf,
            kdims=["t", "f"],
            vdims="logpower"
        ).groupby("f").overlay().opts(width=1200, height=300),
        rolling_window=window
    )

@pn.depends(rang=range_slider.param.value_throttled)
def plot_raw_ts(rang):
    sr = rec.get_sampling_frequency()
    
    return hv.operation.datashader.datashade(
        hv.Curve(
            rec.get_traces(channel_ids=[channel], start_frame=sr*rang[0], end_frame=sr*rang[1]).flatten()
        ),
        aggregator="any"
    ).opts(width=1200, height=300)
    
pn.Column(
    window_slider,freq_slider_1, freq_slider_2, freq_slider_3,range_slider,
    plot_ts,
    plot_raw_ts
)

# TODOs:
- Does phase vary systematically with frequency???
- Does the log power increase with time over the nzight??
- Observation: these birds start singing around 6, before the lights turn on.
- Possibly add spikes for when song occurs 
- Possibly add timerange slider